In [1]:
#Import dependencies
import pandas as pd
import numpy as np

In [2]:
#Create list of leagues and years to retrieve files
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']
years = ['2018', '2017', '2016']

print("Names of dataframes:")

#Use loop to open csvs and read data into dataframes
for year in years:
    for league in leagues:
        path = f"newdata/{league}_{year}.csv"
        
        #Use globals to name dataframe variables after league and year, create league column and drop unneeded columns
        globals()[f"{league}{year[2:]}_df"] = pd.read_csv(path, encoding='utf-8')
        globals()[f"{league}{year[2:]}_df"]['league'] = league
        globals()[f"{league}{year[2:]}_df"].drop(columns=['yellow_cards', 'red_cards', 'position',
                                                          'xGChain', 'xGBuildup', 'season'], inplace=True)
        
        #Print names of dataframes
        print(f"{league}{year[2:]}_df")

Names of dataframes:
EPL18_df
La_liga18_df
Bundesliga18_df
Serie_A18_df
Ligue_118_df
EPL17_df
La_liga17_df
Bundesliga17_df
Serie_A17_df
Ligue_117_df
EPL16_df
La_liga16_df
Bundesliga16_df
Serie_A16_df
Ligue_116_df


In [3]:
#Create dataframes of players by year

for year in years:
    for league in leagues:
        if league == 'EPL':
            pass
        else:
            globals()[f"EPL{year[2:]}_df"] = globals()[f"EPL{year[2:]}_df"].append(globals()[f"{league}{year[2:]}_df"])

In [4]:
#Create dataframe with all leagues and years 
xall_df = EPL18_df.merge(EPL17_df, on='id', how='outer', suffixes=('_18', '_17'))
xall_df = xall_df.merge(EPL16_df, on='id', how='outer')

#Put all names in one column
#Index of the name columns from the 2017 and 2016 seasons
for name_col_index in [13, 25]:
    
    #Create list of indexes of players who did not play in top league in 2018
    pl_name_list = xall_df.index[xall_df.isna()['player_name_18'] == True].tolist()
    
    #Use loop to add 2017 names and then 2016 names
    for nan_index in pl_name_list:
        xall_df.iloc[nan_index, 1] = xall_df.iloc[nan_index, name_col_index]

#Fill NaN columns with 0
xall_df.fillna(0, inplace=True)

#Drop duplicate columns
xall_df.drop(columns=['player_name_17', 'player_name'], inplace=True)

#Rename columns for 2016-17 season
xall_df.rename(columns={'games': 'games_16',
                       'time': 'time_16',
                       'goals': 'goals_16',
                       'xG': 'xG_16',
                       'assists': 'assists_16',
                       'xA': 'xA_16',
                       'shots': 'shots_16',
                       'key_passes': 'key_passes_16',
                       'team_title': 'team_title_16',
                       'npg': 'npg_16',
                       'npxG': 'npxG_16',
                       'league': 'league_16',
                       'player_name_18': 'player_name'}, inplace=True)

In [5]:
#Write dataframe to csv
xall_df.to_csv('cleandata/all_xg.csv', encoding='utf-8')

In [6]:
#Make a copy of the dataframe to remove duplicates
nodup_df = xall_df.copy()

#Get indexes of duplicates
dup_list = nodup_df.index[nodup_df.duplicated(subset='id') == True].tolist()

#Add up player stats for when the switched leagues midseason
for dup in dup_list:
    if nodup_df.iloc[dup - 1, 10] != nodup_df.iloc[dup, 10]:
        nodup_df.iloc[dup, 2:13] = nodup_df.iloc[dup - 1, 2:13] + nodup_df.iloc[dup, 2:13]
    elif nodup_df.iloc[dup - 1, 22] != nodup_df.iloc[dup, 22]:
        nodup_df.iloc[dup, 14:25] = nodup_df.iloc[dup - 1, 14:25] + nodup_df.iloc[dup, 14:25]
    elif nodup_df.iloc[dup - 1, 34] != nodup_df.iloc[dup, 34]:
        nodup_df.iloc[dup, 26:37] = nodup_df.iloc[dup - 1, 26:37] + nodup_df.iloc[dup, 26:37]
        
#Drop duplicates and reset the index        
nodup_df.drop_duplicates(subset='id', keep='last', inplace=True)
nodup_df.reset_index(inplace=True, drop=True)

In [7]:
#Write dataframe to csv
nodup_df.to_csv('cleandata/nodup_xg.csv', encoding='utf-8')